# ST7 Planification quotidienne d’une équipe mobile

## Formulation du problème

### Notation
$ V $ : Nombre de tâches. (V pour vertex)
$ T $ : Nombre de techniciens.

## Variables de décisions
$ X \in R_{V + 1 \times V + 1}$

In [3]:
# module imports
import pandas as pd
import numpy as np
from gurobipy import *
from datetime import datetime
from math import radians, cos, sin, asin, sqrt

In [4]:
# setting file paths
path_bordeaux = "./data/InstancesV1/InstanceBordeauxV1.xlsx"
path_finland = "./data/InstancesV1/InstanceFinlandV1.xlsx"
path_guinea_golf = "./data/InstancesV1/InstanceGuineaGolfV1.xlsx"
path_italy = "./data/InstancesV1/InstanceItalyV1.xlsx"
path_poland = "./data/InstancesV1/InstancePolandV1.xlsx"

path_current = path_finland

In [5]:
df_employees = pd.read_excel(path_current, sheet_name = "Employees")
df_employees.set_index("EmployeeName")

,Latitude,Longitude,Skill,Level,WorkingStartTime,WorkingEndTime
EmployeeName,,,,,,
Aleksi,61.578308,25.210354,Plumbing,1,8:00am,6:00pm
Livia,61.578308,25.210354,Plumbing,2,8:00am,6:00pm
Oskar,61.578308,25.210354,Plumbing,2,8:00am,6:00pm
Sebastian,61.578308,25.210354,Plumbing,1,8:00am,6:00pm


In [6]:
# definition of Employee class
class Employee:

    list = [] # initialized to empty list
    count = 0
    speed = 50 # unit: km/h

    def __init__(self, name: str, latitude: float, longitude: float, skill: str, level: int, start_time, end_time):
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
        self.skill = skill
        self.level = level
        self.start_time = start_time
        self.end_time = end_time
        Employee.count += 1
        Employee.list.append(self)

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        return self.name == other.name

    def __str__(self):
        return self.name

    def __repr__(self):
        return f"Employee(name={self.name}, " \
               f"position=[{self.longitude}, {self.latitude}], " \
               f"skill={self.skill}," \
               f"level={self.level}," \
               f"available=[{self.start_time_string()}, {self.end_time_string()}] )"

    def start_time_string(self):
        return self.start_time.strftime('%I:%M%p')

    def end_time_string(self):
        return self.end_time.strftime('%I:%M%p')

In [7]:
# definition of Task class
class Task:

    list = []
    count = 0
    distance : np.array = None
    __is_initialized = False

    def __init__(self, task_id, latitude, longitude, duration, skill, level, start_time, end_time):
        if Task.__is_initialized:
            raise Exception("Cannot instantiate new task after initializing the distance matrix")
        self.id = task_id
        self.latitude = latitude
        self.longitude = longitude
        self.duration = duration
        self.skill = skill
        self.level = level
        self.start_time = start_time
        self.end_time = end_time
        Task.list.append(self)
        Task.count += 1

    @staticmethod
    def calculate_distance(task1, task2):
        lon1, lat1 = radians(task1.longitude), radians(task1.latitude)
        lon2, lat2 = radians(task2.longitude), radians(task2.latitude)

        # Haversine formula
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * asin(sqrt(a))
        r = 6371
        return c * r

    @classmethod
    def initialize_distance(cls):
        if cls.__is_initialized:
            raise Exception("Distance has already been initialized")
        cls.__is_initialized = True
        cls.distance = np.zeros((cls.count, cls.count), dtype=np.float64)

        for i in range(cls.count):
            for j in range(i):
                task_i, task_j = cls.list[i], cls.list[j]
                cls.distance[i, j] = cls.distance[j, i] = cls.calculate_distance(task_i, task_j)

    def __hash__(self):
        return hash(self.id)

    def __eq__(self, other):
        return self.id == other.id

In [8]:
# reading dataframe into python objects
for index, row in df_employees.iterrows():
    # parse the start time and end time into datetime object
    start_time = datetime.strptime(row["WorkingStartTime"], '%I:%M%p')
    end_time = datetime.strptime(row["WorkingEndTime"], '%I:%M%p')

    Employee(row["EmployeeName"],
                        row["Latitude"],
                        row["Longitude"],
                        row["Skill"],
                        row["Level"],
                        start_time,
                        end_time)

In [9]:
# test on employee object
first_employee = Employee.list[0]

In [10]:
# Testing printing function
print(first_employee)

Aleksi


In [13]:
# Testing representation function for debugging
first_employee

Employee(name=Aleksi, position=[25.2103536, 61.578308], skill=Plumbing,level=1,available=[08:00AM, 06:00PM] )

In [ ]:
model = Model("V1")

# variables de decision
